# LSTM, GRU 이론
https://velog.io/@cha-suyeon/DL-Long-Short-Term-MemoryLSTM-and-GRU

# windowed_dataset

1. expand_dim()

tensorflow의 expand_dim은 차원을 늘려준다.

In [1]:
import tensorflow as tf
import numpy as np

x = np.arange(20)
print(x.shape)

tf.expand_dims(x, 1).shape

(20,)


TensorShape([20, 1])

2. batch 메서드 

모델에 학습시킬 때 batch_size를 지정하여 batch_size만큼 데이터를 읽어들여 학습시킬 때 유용한 method이다. 이미지와 같은 큰 사이즈는 memory에 한 번에 올라가지 못하기 때문에 batch를 나누어 학습시키기도 한다. 또한 모델이 weight를 업데이트 할 때, 1개의 batch가 끝나고 난 후 업데이트를 하게 되는데, 업데이트 빈도를 조절하는 효과도 있다. 

옵션

+ drop_remainder : 마지막 남은 데이터를 drop할 것인지 여부

In [2]:
ds = tf.data.Dataset.range(8)

# 각 batch 확인
for d in ds.batch(3, drop_remainder=True):
    print(d)

tf.Tensor([0 1 2], shape=(3,), dtype=int64)
tf.Tensor([3 4 5], shape=(3,), dtype=int64)


6,7이 batch로 출력되어야 하지만 drop_remainder=True 옵션 때문에 6,7을 drop하였다.

In [3]:
ds = tf.data.Dataset.range(8)

# 각 batch 확인
for d in ds.batch(5, drop_remainder=True):
    print(d)

tf.Tensor([0 1 2 3 4], shape=(5,), dtype=int64)


size가 5가 되지 않는 window는 drop되었다.

3. window : Time Series 데이터셋 생성에 유용

인자

+ window : 그룹화 할 윈도우 크기(갯수)
+ drop_remainder : 남은 부분을 drop할 것인지 여부
+ shift : 1번 반복할때마다 몇 개씩 이동할 것인지

In [ ]:
# window의 크기는 5, sift=1, drop_remainder=False인 경우

ds = tf.data.Dataset.range(10)
ds = ds.window(5, shift=1, drop_remainder=False)
for d in ds:
    print(list(d.as_numpy_iterator()))

[0, 1, 2, 3, 4]
[1, 2, 3, 4, 5]
[2, 3, 4, 5, 6]
[3, 4, 5, 6, 7]
[4, 5, 6, 7, 8]
[5, 6, 7, 8, 9]
[6, 7, 8, 9]
[7, 8, 9]
[8, 9]
[9]


In [45]:
# window의 크기는 5, sift=1, drop_remainder=True인 경우

ds = tf.data.Dataset.range(10)
ds = ds.window(5, shift=1, drop_remainder=True)
for d in ds:
    print(list(d.as_numpy_iterator()))

[0, 1, 2, 3, 4]
[1, 2, 3, 4, 5]
[2, 3, 4, 5, 6]
[3, 4, 5, 6, 7]
[4, 5, 6, 7, 8]
[5, 6, 7, 8, 9]


4. flat_map : dataset에 함수를 apply해주고, 결과를 faltten하게 펼쳐준다.

In [11]:
# lambda 함수를 통해 3개의 batch를 읽어들인 뒤 flatten된 리턴값을 받는다.

ds = tf.data.Dataset.range(10)
ds = ds.window(5, shift=1, drop_remainder=True)
ds = ds.flat_map(lambda w: w.batch(3))
for d in ds:
    print(d) 

tf.Tensor([0 1 2], shape=(3,), dtype=int64)
tf.Tensor([3 4], shape=(2,), dtype=int64)
tf.Tensor([1 2 3], shape=(3,), dtype=int64)
tf.Tensor([4 5], shape=(2,), dtype=int64)
tf.Tensor([2 3 4], shape=(3,), dtype=int64)
tf.Tensor([5 6], shape=(2,), dtype=int64)
tf.Tensor([3 4 5], shape=(3,), dtype=int64)
tf.Tensor([6 7], shape=(2,), dtype=int64)
tf.Tensor([4 5 6], shape=(3,), dtype=int64)
tf.Tensor([7 8], shape=(2,), dtype=int64)
tf.Tensor([5 6 7], shape=(3,), dtype=int64)
tf.Tensor([8 9], shape=(2,), dtype=int64)


In [12]:
# lambda 함수를 통해 5개의 batch를 읽어들인 뒤 flatten된 리턴값을 받는다.

ds = tf.data.Dataset.range(10)
ds = ds.window(5, shift=1, drop_remainder=True)
ds = ds.flat_map(lambda w: w.batch(5))
for d in ds:
    print(d) 

tf.Tensor([0 1 2 3 4], shape=(5,), dtype=int64)
tf.Tensor([1 2 3 4 5], shape=(5,), dtype=int64)
tf.Tensor([2 3 4 5 6], shape=(5,), dtype=int64)
tf.Tensor([3 4 5 6 7], shape=(5,), dtype=int64)
tf.Tensor([4 5 6 7 8], shape=(5,), dtype=int64)
tf.Tensor([5 6 7 8 9], shape=(5,), dtype=int64)


5. shuffle 

suffle은 Dataset을 섞어주는 역할을 하며, 반드시 학습 전에 shuffle을 통해 적절하게 Dataset을 섞어주어야 한다.

In [22]:
# 셔플을 해주지 않은 경우
import numpy as np

ds = tf.data.Dataset.from_tensor_slices(np.arange(10))
for d in ds:
    print(d)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


In [26]:
ds = tf.data.Dataset.from_tensor_slices(np.arange(10)).shuffle(buffer_size=5)

for d in ds:
    print(d)

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)


+ shuffle의 인자 buffer_size

데이터세트는 beffer_size 요소로 버퍼를 채운 다음에 버퍼에서 요소를 무작위로 샘플링하여 선택한 요소를 새 요소로 바꾼다. 완벽한 셔플링을 위해서는 데이터 세트의 전체 크기보다 크거나 같은 버퍼크기가 필요하다. 

In [ ]:
# 예시
dataset = dataset.shuffle(buffer_size=1000) # 1000이 옳은가?
dataset = dataset.batch(100)

위코드를 실행하면 전체 데이터의 처음 1000개의 데이터를 buffer에 가져올것이고 그 다음에 1000개 중에 랜덤하게 100개를 pick할것

6. map

Dataset 전체에 함수를 매핑한다. Time Series Dataset을 만드려는 경우, train/label값을 분류하는 용도로 활용할 수 있다.

In [9]:
window_size=5
ds = tf.data.Dataset.range(10)
ds = ds.window(window_size, shift=1, drop_remainder=True)
ds = ds.flat_map(lambda w: w.batch(window_size))
ds = ds.shuffle(10)

ds = ds.map(lambda x: (x[:-1], x[-1:]))
for x, y in ds:
    print('train set: {}'.format(x))
    print('label set: {}'.format(y))

train set: [1 2 3 4]
label set: [5]
train set: [0 1 2 3]
label set: [4]
train set: [2 3 4 5]
label set: [6]
train set: [4 5 6 7]
label set: [8]
train set: [5 6 7 8]
label set: [9]
train set: [3 4 5 6]
label set: [7]


# Conv1D

(케라스 1convid layer 번역 (https://keras.io/api/layers/convolution_layers/convolution1d/))

+ filters : 정수, 출력 공간의 차원(즉, 컨볼루션의 출력 필터 수).

+ kernel_size : 1D 컨볼루션 창의 길이를 지정하는 정수 또는 단일 정수의 튜플/목록.

+ padding : 
     1. "valid" : 패딩 없음을 의미
     2. "same" : 결과적으로 출력이 입력과 동일한 높이/너비 치수를 갖도록 입력의 왼쪽/오른쪽 또는 위/아래에 균일하게 0이 채워진다. 
     3. "causal" : 인과 관계(확장된) 컨볼루션이 발생합니다(예: output[t] 에 의존하지 않음 ) input[t+1:]. 모델이 시간 순서를 위반하지 않아야 하는 시간 데이터를 모델링할 때 유용하다.


+ activation : 사용할 활성화 함수이다. 아무것도 지정하지 않으면 활성화 함수가 사용되지 않는다. (https://keras.io/ko/activations/ 참조)

# Model compile()

complie() 메소드는 다음 세개의 인자를 입력으로 받는다.
+ 정규화기 (optimizaer)
  + 훈련과정을 설정한다. 즉, 최적화 알고리즘 설정을 의미한다.
  + adam, sgd, rmsprop, adagrad 등이 잇다.
+ 손실 함수 (loss function)
  + 모델이 최적화에 사용되는 목적 함수이다.
  + mse, categorical_crossentropy, binary_crossentropy 등이 있다.
+ 평가지표 (metric)
  + 훈련을 모니터링 하기 위해 사용된다.
  + 분류에서는 accuracy, 회귀에서는 mse, rmse, r2, mae, mspe, mape, msle 등이 있다.
  + 사용자가 메트릭을 정의해서 사용할 수도 있다. 